In [1]:
from parser import *

import itertools
import random
import pickle
import numpy as np
from operator import itemgetter
from collections import Counter
from itertools import izip
import scipy
# import scipy.spatial.distance
from numpy.linalg import svd
from sklearn.cluster import AffinityPropagation
from collections import defaultdict
from sklearn.feature_selection import RFE
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectFpr, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

In [2]:
from summanlp_textrank import commons, graph, keywords, pagerank_weighted, \
                  summarizer, textrank, textcleaner, textrank_runtime_error

In [3]:
%load_ext autoreload

In [4]:
# To reload: 
%autoreload

In [5]:
inputXML = "sample_1"
# inputXML = "sample_rawXML"
summaryFolder = "sample_summary"
bodyFolder = "sample_body"

In [6]:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# PARSE:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
documents, labels = parse_xml_folder(inputXML, summaryFolder, bodyFolder, writeOption='none')
surfaceFeatures = process_data(documents)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
STAGE [1] -- PARSING XML -- from sample_1 ...
STAGE [2] -- PROCESSING DATA -- (tokenizing/tagging/stopwords/extracting) ...


In [7]:
def featurize(documents, surfaceFeatures):
    print "STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ..."
    features = []
    for docIndex, doc in enumerate(documents):
        documentFeatures = extract_document_wide_features(doc)
        documentFeatures.append(surfaceFeatures[docIndex])
        features += [ counter_sum(fl) for fl in izip(*documentFeatures) ]
    return features

In [8]:
def extract_document_wide_features(document):
    documentFeatures = []

    documentFeatures.append(textrank_keyphrase(document))
    documentFeatures.append(lexrank_keyphrase(document))
    documentFeatures.append(textrank_keyword(document))

    return documentFeatures

In [9]:
def counter_sum(counterTuple):
    counterSum = Counter()
    for ele in counterTuple:
        counterSum += ele
    return counterSum

In [10]:
def textrank_keyphrase(text):

    # Creates the graph and calculates the similarity coefficient for every pair of nodes.
    graph = commons.build_graph([ syntacticUnit for syntacticUnit in text])
    summarizer._set_graph_edge_weights(graph)
    # Remove all nodes with all edges weights equal to zero.
    commons.remove_unreachable_nodes(graph)

    # Ranks the tokens using the PageRank algorithm. Returns dict of sentence -> score
    pagerank_scores = summarizer._pagerank(graph)

    # Adds the summa scores to the sentence objects.
    # summarizer._add_scores_to_sentences(sentences, pagerank_scores)

    results = []
    for su in text:
        score = (1-pagerank_scores[su.label, su.index]) if (su.label, su.index) in pagerank_scores.keys() else 0.0
        results.append(Counter({ 'TEXTRANK_SCORE': score }))
    return results

In [11]:
def textrank_keyword(text):
    txt = u' '.join([ su.text for su in text ])
    # Gets a dict of word -> lemma
    tokens = textcleaner.clean_text_by_word(text, 'english')
    split_text = list(textcleaner.tokenize_by_word(txt))

    # Creates the graph and adds the edges
    graph = commons.build_graph(keywords._get_words_for_graph(tokens))
    keywords._set_graph_edges(graph, tokens, split_text)
    del split_text # It's no longer used
    commons.remove_unreachable_nodes(graph)

    # # Ranks the tokens using the PageRank algorithm. Returns dict of lemma -> score
    pagerank_scores = keywords._pagerank_word(graph)
    extracted_lemmas = keywords._extract_tokens(graph.nodes(), pagerank_scores, 0.2, None)
    lemmas_to_word = keywords._lemmas_to_words(tokens)
    keyWords = keywords._get_keywords_with_score(extracted_lemmas, lemmas_to_word)
    # # text.split() to keep numbers and punctuation marks, so separeted concepts are not combined
    combined_keywords = keywords._get_combined_keywords(keyWords, txt.split())
    kw_scores = keywords._format_results(keyWords, combined_keywords, False, True)

    results = [ Counter({ 'TEXTRANK_KEYWORD_SCORE': keyword_mean_score(su.basic, kw_scores) }) for su in text ]
    return results

In [18]:
def keyword_mean_score(sentence, wordScores):
    totalScore = sum([ s for w, s in wordScores if w in sentence ])
#     print sentence.split()
    if not sentence.split(): return 0.0
    return totalScore / len(sentence.split())

In [13]:
def lexrank_keyphrase(text):
    results = []
    for i in range(len(text)):
        results.append(Counter({ 'LEXRANK_SCORE': 0.0 }))
    return results

In [14]:
def train_classifier(features, labels):
    print "STAGE [4] -- TRAINING MODEL -- Logistic Regression ..."
    print '<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>'
    vectorizer = DictVectorizer(sparse=False)
    feature_matrix = vectorizer.fit_transform(features) # Features = List of counters
    mod = LogisticRegression(fit_intercept=True, intercept_scaling=1, class_weight='auto')
    mod.fit_transform(feature_matrix, labels)
    return mod, feature_matrix, vectorizer

In [15]:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# TRAIN:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
features = featurize(documents, surfaceFeatures)
# del surfaceFeatures
model, featMatrix, vectorizer = train_classifier(features, labels)

scores = cross_val_score(model, featMatrix, labels, scoring="f1_macro") # accuracy, f1, log_loss
print model.coef_
predictions = model.predict(featMatrix)
print metrics.classification_report(labels, predictions)

STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ...
[u'jrr']
[u'tolkien', u's', u'the', u'lord', u'of', u'the', u'rings', u'is', u'a', u'american', u'fantasy', u'film', u'directed', u'by', u'ralph', u'bakshi']
[u'it', u'uses', u'a', u'hybrid', u'of', u'traditional', u'cel', u'animation', u'and', u'rotoscoped', u'live', u'action', u'footage']
[u'it', u'is', u'an', u'adaptation', u'of', u'the', u'first', u'half', u'of', u'the', u'high', u'fantasy', u'epic', u'the', u'lord', u'of', u'the', u'rings', u'by', u'english', u'novelist', u'j', u'r', u'r', u'tolkien']
[u'set', u'in', u'middle', u'earth', u'the', u'film', u'follows', u'a', u'group', u'of', u'hobbits', u'elves', u'men', u'dwarves', u'and', u'wizards', u'who', u'form', u'a', u'fellowship']
[u'they', u'embark', u'on', u'a', u'quest', u'to', u'destroy', u'the', u'one', u'ring', u'made', u'by', u'the', u'dark', u'lord', u'sauron', u'and', u'ensure', u'his', u'destruction']
[u'the', u'film', u'features', u'the', u'voices', u'of', u

In [19]:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# TEST:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
testXMLFolder = "sample_test"
testDocuments, testLabels = parse_xml_folder(testXMLFolder, summaryFolder, bodyFolder, writeOption='none')
testSurfaceFeatures = process_data(testDocuments)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
STAGE [1] -- PARSING XML -- from sample_test ...
STAGE [2] -- PROCESSING DATA -- (tokenizing/tagging/stopwords/extracting) ...


In [20]:
testFeatures = featurize(testDocuments, testSurfaceFeatures)
testFeatureMatrix = vectorizer.transform(testFeatures) # Features = List of counters
# del testSurfaceFeatures

STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ...
[u'john', u'calvin', u'french', u'jean', u'calvin', u'born', u'jehan', u'cauvin', u'july', u'may', u'was', u'an', u'influential', u'french', u'theologian', u'and', u'pastor', u'during', u'the', u'protestant', u'reformation']
[u'he', u'was', u'a', u'principal', u'figure', u'in', u'the', u'development', u'of', u'the', u'system', u'of', u'christian', u'theology', u'later', u'called', u'calvinism']
[u'originally', u'trained', u'as', u'a', u'humanist', u'lawyer', u'he', u'broke', u'from', u'the', u'roman', u'catholic', u'church', u'around']
[u'after', u'religious', u'tensions', u'provoked', u'a', u'violent', u'uprising', u'against', u'protestants', u'in', u'france', u'calvin', u'fled', u'to', u'basel', u'switzerland', u'where', u'he', u'published', u'the', u'first', u'edition', u'of', u'his', u'seminal', u'work', u'the', u'institutes', u'of', u'the', u'christian', u'religion', u'in']
[u'in', u'that', u'year', u'calvin', u'was', u'recr

In [21]:
def evaluate_trained_classifier(model, feature_matrix, labels):
    """Evaluate model, the output of train_classifier, on the test data."""
    print "STAGE [5] -- TESTING -- Logistic Regression ..."
    print '<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>'
    predictions = model.predict(feature_matrix)
    print cross_val_score(model, feature_matrix, labels, scoring="f1_macro")
    return metrics.classification_report(labels, predictions)

In [22]:
print evaluate_trained_classifier(model, testFeatureMatrix, testLabels)

STAGE [5] -- TESTING -- Logistic Regression ...
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
[ 0.47271142  0.39633872  0.40424908]
             precision    recall  f1-score   support

       BODY       0.94      0.66      0.78       607
    SUMMARY       0.06      0.37      0.11        38

avg / total       0.89      0.64      0.74       645



In [23]:
def extract_summary(document):
    summary = []
    for su in document:
        if su.label == 'summary':
            summary.append(su.text)
        else:
            break
    return summary

In [24]:
from heapq import nlargest

def extract_top_ranked(document, featureMatrix, num):
    predictions = model.predict_proba(featureMatrix)
    itr = range(len(document))
    topIndexes = nlargest(num, itr, key=lambda i: predictions[i][1])
    topSentences = [ document[index].text for index in topIndexes]
    return topSentences

In [25]:
featIndex = 0
rouge_gold_summaries = []
rouge_generated_summaries = []
for i, tdoc in enumerate(testDocuments):
    numSentences = len(tdoc)
    rouge_gold_summaries.append(extract_summary(tdoc))
    rouge_generated_summaries.append(extract_top_ranked(tdoc, testFeatureMatrix[featIndex:featIndex+numSentences], 10))
    featIndex += numSentences

#print rouge_gold_summaries
#print rouge_generated_summaries

In [26]:
import pickle

In [27]:
pickle_gold_summaries = {}
pickle_generated_summaries = {}
for i in range(len(rouge_gold_summaries)):
    pickle_gold_summaries[i] = rouge_gold_summaries[i]
    pikcle_generated_summaries = rouge_generated_summaries[i]

In [28]:
pickle.dump( pickle_gold_summaries, open( "pickle_gold_summaries.p", "wb" ) )
pickle.dump( pickle_generated_summaries, open( "pickle_generated_summaries.p", "wb" ) )

In [40]:
from RougeRunner import *

rougeResults = []

for summaryIndex in range(len(rouge_gold_summaries)):
    gold = rouge_gold_summaries[summaryIndex]
    genSum = rouge_generated_summaries[summaryIndex]
    rougeResults.append(compareUsingRouge(gold, genSum))
    
print rougeResults

NameError: name 'rouge_gold_summaries' is not defined